In [18]:
import os
import torch
print("torch module path:", torch.__file__)
print(torch.cuda.is_available())
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score, log_loss

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader, random_split

torch module path: c:\Users\Daria\anaconda3\Lib\site-packages\torch\__init__.py
False


In [19]:

data = pd.read_csv('Data/Final_data/final_schizo_one_scaled')

data = data.iloc[:, 1:]
data.head()

,821,328813,823,817,818,239935,28116,351091,28118,1161942,...,78344,1692,68274,40091,1393,28133,183,29346,349096,Schizophrenia
0,1.439593,-0.405899,-0.592464,-0.294858,-0.629663,-0.388186,0.060858,-0.638728,-0.158473,-0.274041,...,-0.255388,-0.258027,-0.19843,2.305337,-0.153823,-0.106539,-0.177886,-0.184357,-0.063034,0
1,-0.746323,-0.497882,0.149193,-0.318254,-0.611975,-0.337264,-0.599826,0.072958,-0.541310,2.875357,...,-0.255388,-0.258027,-0.19843,-0.209176,-0.153823,-0.106539,-0.177886,-0.184357,-0.077118,1
2,0.491225,0.320783,-0.668681,-0.139988,-0.220985,-0.293335,3.686425,-0.317174,-0.216365,-0.257336,...,-0.255388,-0.258027,-0.19843,-0.209176,-0.153823,-0.106539,-0.177886,0.329359,-0.077118,0
3,0.234604,-0.341134,-0.547374,-0.308704,0.088972,-0.388365,0.077232,1.987442,-0.475624,-0.262770,...,-0.255388,-0.258027,-0.19843,-0.209176,-0.153823,-0.106539,-0.177886,-0.184357,-0.077118,0
4,0.933617,-0.362895,-0.236814,-0.193425,-0.012090,-0.175105,0.195748,-0.186020,0.265365,-0.033088,...,-0.255388,-0.258027,-0.19843,-0.209176,-0.153823,-0.106539,-0.177886,-0.184357,-0.077118,1


In [20]:
data.shape

(434, 557)

In [21]:
class Model(nn.Module):
    def __init__(self, input_size, h1, h2, h3, output_size = 2):
        super(Model, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, h1),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(h2, h3),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(h3, output_size) 
        )
    def forward(self, x):
        return self.net(x)


In [22]:
class SklearnWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, input_size=10, h1=32, h2=16, h3=16, lr=0.01, num_epochs=100):
        self.input_size = input_size
        self.h1 = h1
        self.h2 = h2
        self.h3 = h3
        self.lr = lr
        self.num_epochs = num_epochs
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None

    def fit(self, X, y):
        dataset = TensorDataset(X, y)
        
        val_size = int(0.2 * len(dataset))
        train_size = len(dataset) - val_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)

        self.model = Model(self.input_size, self.h1, self.h2, self.h3).to(self.device)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        criterion = nn.CrossEntropyLoss()

        best_loss = float('inf')
        patience = 20
        wait = 0
        best_state = None

        for epoch in range(self.num_epochs):
            self.model.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                pred = self.model(xb)
                loss = criterion(pred, yb)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        
            self.model.eval()
            val_loss = 0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(self.device), yb.to(self.device)
                    pred = self.model(xb)
                    val_loss += criterion(pred, yb).item()
                    

            val_loss /= len(val_loader)

            if val_loss < best_loss:
                best_loss = val_loss
                best_state = self.model.state_dict()
                wait = 0
            else:
                wait += 1
                if wait >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        #if best_state:
         #   self.model.load_state_dict(best_state)
        
        self.best_state_dict = best_state
   
        if self.best_state_dict is not None:
            self.model.load_state_dict(self.best_state_dict)

        return self


    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X)
        return outputs.argmax(dim=1).cpu().numpy()


In [23]:
param_grid = {
    'h1': [32, 64, 96, 128, 160],
    'h2': [16, 32, 64, 96],
    'h3': [16, 32, 64, 96],
    'lr': [0.0001, 0.0005],
    'num_epochs': [40, 60, 100, 120, 160, 200]
}

In [24]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')
    #'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True) 
    #check loss calc
}

In [25]:
X = data.drop('Schizophrenia', axis = 1)
y = data['Schizophrenia']

X = X.values
y = y.values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#X_train = torch.FloatTensor(X_train)
#X_test = torch.FloatTensor(X_test)

#y_train = torch.LongTensor(y_train)
#y_test = torch.LongTensor(y_test)

In [26]:

nested_scores = []

outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in outer_cv.split(X, y):
    X_train_cv = torch.FloatTensor(X[train_idx])
    y_train_cv = torch.LongTensor(y[train_idx])
    X_test_cv = torch.FloatTensor(X[test_idx])
    y_test_cv = y[test_idx]

    clf = GridSearchCV(SklearnWrapper(input_size=X.shape[1]), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    clf.fit(X_train_cv, y_train_cv)

    preds = clf.predict(X_test_cv)
    acc = accuracy_score(y_test_cv, preds)
    nested_scores.append(acc)
    print(f"Fold Accuracy: {acc:.4f}")

print(f"Nested CV Mean Accuracy: {np.mean(nested_scores):.4f}")


Early stopping at epoch 35
Fold Accuracy: 0.9195
Early stopping at epoch 52
Fold Accuracy: 0.8851
Early stopping at epoch 35
Fold Accuracy: 0.8966
Early stopping at epoch 38
Fold Accuracy: 0.8506
Early stopping at epoch 35
Fold Accuracy: 0.8023
Nested CV Mean Accuracy: 0.8708


In [27]:
print(clf.best_estimator_.model)

Model(
  (net): Sequential(
    (0): Linear(in_features=556, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=64, out_features=16, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=16, out_features=96, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=96, out_features=2, bias=True)
  )
)


In [28]:
#m = clf.best_state_dict

In [29]:
best_model = clf.best_estimator_

In [30]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = best_model.model.forward(data)

    print(f'{i+1}.)  {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

    # Correct or not
    if y_val.argmax().item() == y_test[i]:
      correct +=1

print(f'We got {correct} correct!')
accuracy = correct / list(y_test.shape)[0]
print(f'Accuracy is: {accuracy}')

1.)  tensor([ 4.1434, -4.7077]) 	 0 	 0
2.)  tensor([-5.3921,  5.2129]) 	 1 	 1
3.)  tensor([-5.5615,  5.5170]) 	 1 	 1
4.)  tensor([-4.6248,  4.4974]) 	 1 	 1
5.)  tensor([-2.1428,  1.6851]) 	 1 	 1
6.)  tensor([ 6.2068, -7.2347]) 	 0 	 0
7.)  tensor([ 4.0744, -4.7090]) 	 0 	 0
8.)  tensor([ 2.7329, -3.0971]) 	 0 	 0
9.)  tensor([ 4.7679, -5.5309]) 	 0 	 0
10.)  tensor([ 6.3512, -7.4028]) 	 0 	 0
11.)  tensor([ 3.3522, -3.8782]) 	 0 	 0
12.)  tensor([-6.9197,  6.6387]) 	 1 	 1
13.)  tensor([ 3.3071, -3.8212]) 	 0 	 0
14.)  tensor([-9.5390,  9.1712]) 	 1 	 1
15.)  tensor([ 4.7842, -5.5651]) 	 0 	 0
16.)  tensor([-13.1907,  12.6885]) 	 1 	 1
17.)  tensor([-5.5930,  5.4075]) 	 1 	 1
18.)  tensor([-2.6337,  2.4759]) 	 1 	 1
19.)  tensor([ 5.1786, -6.0549]) 	 0 	 0
20.)  tensor([-8.9167,  8.5428]) 	 1 	 1
21.)  tensor([-8.4261,  8.1530]) 	 1 	 1
22.)  tensor([ 6.9398, -8.1106]) 	 0 	 0
23.)  tensor([-3.8820,  3.7375]) 	 1 	 1
24.)  tensor([ 1.7109, -1.9897]) 	 0 	 0
25.)  tensor([-4.9289, 

In [31]:
print(list(y_test.shape))

[87]


In [32]:
#torch.save(best_model.model.state_dict(), 'Models/3_layer_nn_96_64_64_with_dropout')

In [33]:
#model = Model(input_size=X_train.shape[1], h1=160, h2=16, h3=16, output_size=2)
#model.load_state_dict(torch.load('Models/3_layer_nn_160_16_16'))
#model.eval()

In [34]:
'''correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    print(f'{i+1}.)  {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

    # Correct or not
    if y_val.argmax().item() == y_test[i]:
      correct +=1

print(f'We got {correct} correct!')'''

"correct = 0\nwith torch.no_grad():\n  for i, data in enumerate(X_test):\n    y_val = model.forward(data)\n\n    print(f'{i+1}.)  {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')\n\n    # Correct or not\n    if y_val.argmax().item() == y_test[i]:\n      correct +=1\n\nprint(f'We got {correct} correct!')"